In [1]:
import lal
import json
import numpy as np
import sys
sys.path.append('../src')
sys.path.append('../scripts')
import standard_gwtransf as gw
import os
import pycbc
import pycbc.noise
import pycbc.psd
from pycbc.frame import query_and_read_frame
from pycbc.types import timeseries
from pycbc.psd import inverse_spectrum_truncation, interpolate
from pycbc.waveform import get_fd_waveform, get_td_waveform
import pycbc.filter.matchedfilter as mfilter
from pycbc import frame as Fr
import scipy.signal
import imrtgrutils_final as tgr
import nr_fits as nr
import glob
import matplotlib.pyplot as plt
import lalsimulation

### Read in details of the LVC-NR catalog, and separate out the waveforms from the SXS catalog

In [2]:
f_lvcnr = open('/home/abhirup/ligo-nr-data/lvcnr-lfs.json', 'r')
datastore = json.load(f_lvcnr)
f_lvcnr.close()

sxs_list = []

for idx in range(len(datastore)):
    if datastore[idx]['NR-group'] == 'SXS':
        sxs_list.append(idx)

### Read in science quality data stretches from O1 and O2
For O2, we have chosen coincident segments for HL and HLV configurations, separately.

In [3]:
o1_L1H1_start_times, o1_L1H1_end_times, o1_L1H1_stretches = np.loadtxt('../data/systematics_error_characterisation/o1_L1H1_quality_segments.txt', unpack=True)
o2_L1H1_start_times, o2_L1H1_end_times, o2_L1H1_stretches = np.loadtxt('../data/systematics_error_characterisation/o2_L1H1_quality_segments.txt', unpack=True)
o2_L1H1V1_start_times, o2_L1H1V1_end_times, o2_L1H1V1_stretches = np.loadtxt('../data/systematics_error_characterisation/o2_L1H1V1_quality_segments.txt', unpack=True)

### Specifying the parameters of the binary to be simulated using a specific SXS-NR waveform
The SXS waveform has information about the mass ratio, q or (m1, m2) (normalised to a total mass M = 1 M_sun) and the spins: (s1x, s1y, s1z, s2x, s2y, s2z). We choose the other parameters (iota, pol, ra, dec, snr) and normalise to a total mass M = 100 M_sun.

In [4]:
idx = np.random.choice(sxs_list)
name = datastore[idx]['name'].replace(':','_')
path = datastore[idx]['path']

print '... chose:', name, idx, '/home/abhirup/ligo-nr-data/lvcnr-lfs/%s'%path

M = np.random.uniform(20,160) # total mass chosen random from distribution uniform in [20,160] M_sun
m1, m2 = datastore[idx]['mass1']*M, datastore[idx]['mass2']*M
f_low = 25.
f_ref = f_low
s1x,s1y,s1z,s2x,s2y,s2z = lalsimulation.SimInspiralNRWaveformGetSpinsFromHDF5File(f_ref, M, ('/home/abhirup/ligo-nr-data/lvcnr-lfs/%s'%path).encode('utf-8'))
iota = np.arccos(np.random.uniform(-1., 1.))
pol = np.random.uniform(0., np.pi)
ra = np.random.uniform(0., 2.*np.pi)
dec = np.random.uniform(-np.pi/2., np.pi/2.)
snr = np.random.uniform(8,25)   # choose a random SNR uniformly distributed in [8,25]

print '... chose: M:%.2f, SNR:%.2f'%(M,snr)

... chose: SXS_BBH_0169 340 /home/abhirup/ligo-nr-data/lvcnr-lfs/SXS/SXS_BBH_0169_Res4.h5


RuntimeError: Invalid argument

### Choosing an observing run and detector configuration at random

In [ ]:
if obs_run == 'o1':
    if det_config == 'HL':
          start_times, end_times, stretches = o1_L1H1_start_times, o1_L1H1_end_times, o1_L1H1_stretches
    elif det_config == 'HLV':
          start_times, end_times, stretches = o2_L1H1V1_start_times, o2_L1H1V1_end_times, o2_L1H1V1_stretches
elif obs_run == 'o2':
    start_times, end_times, stretches = o2_L1H1_start_times, o2_L1H1_end_times, o2_L1H1_stretches

# choose a random science quality data stretch more than 128 seconds and place the trigger at the centre of that data chunk
index = np.random.randint(len(stretches))      # choose a random science quality data stretch
if stretches[index] > 128:                     # ensure it is greater than 128 seconds long

    # choose a 128 second window in that stretch and place the trigger time at the centre
    gps_start_time = np.random.randint(start_times[index],end_times[index]-128)
    gps_end_time = int(gps_start_time + 128)
    geocentric_end_time = np.ceil(gps_start_time + (gps_end_time - gps_start_time)/2.)

    print '... chose:', det_config, obs_run, gps_start_time, gps_end_time, geocentric_end_time

In [ ]:
tag_sxs = name + '_' + obs_run + '_' + det_config + '_' + str(int(geocentric_end_time)) + '_sxs'
tag_imrppv2 = name + '_' + obs_run + '_' + det_config + '_' + str(int(geocentric_end_time)) + '_imrppv2'

pycbc_generate_hwinj_command_sxs = "pycbc_generate_hwinj --numrel-data /home/abhirup/ligo-nr-data/lvcnr-lfs/%s --approximant NR_hdf5 --waveform-low-frequency-cutoff 25 --mass1 %f --mass2 %f --spin1x %f --spin1y %f --spin1z %f --spin2x %f --spin2y %f --spin2z %f --inclination %f --polarization %f --ra %f --dec %f --instruments %s --sample-rate H1:16384 L1:16384 V1:16384 --taper TAPER_START --network-snr %f --low-frequency-cutoff 25.0 --high-frequency-cutoff 1024.0 --psd-model H1:aLIGOZeroDetHighPower L1:aLIGOZeroDetHighPower V1:AdVDesignSensitivityP1200087 --geocentric-end-time %d --gps-start-time %d --gps-end-time %d --strain-high-pass 1 --psd-output H1:%s_H1_psd.txt L1:%s_L1_psd.txt V1:%s_V1_psd.txt --tag %s"%(path, m1, m2, s1x, s1y, s1z, s2x, s2y, s2z, iota, pol, ra, dec, ifo_map[det_config], snr, geocentric_end_time, gps_start_time, gps_end_time, tag_sxs, tag_sxs, tag_sxs, tag_sxs)
pycbc_generate_hwinj_command_imrppv2 = "pycbc_generate_hwinj --approximant IMRPhenomPv2 --order pseudoFourPN --waveform-low-frequency-cutoff 25 --mass1 %f --mass2 %f --spin1x %f --spin1y %f --spin1z %f --spin2x %f --spin2y %f --spin2z %f --inclination %f --polarization %f --ra %f --dec %f --instruments %s --sample-rate H1:16384 L1:16384 V1:16384 --taper TAPER_START --network-snr %f --low-frequency-cutoff 25.0 --high-frequency-cutoff 1024.0 --psd-model H1:aLIGOZeroDetHighPower L1:aLIGOZeroDetHighPower V1:AdVDesignSensitivityP1200087 --geocentric-end-time %d --gps-start-time %d --gps-end-time %d --strain-high-pass 1 --psd-output H1:%s_H1_psd.txt L1:%s_L1_psd.txt V1:%s_V1_psd.txt --tag %s"%(m1, m2, s1x, s1y, s1z, s2x, s2y, s2z, iota, pol, ra, dec, ifo_map[det_config], snr, geocentric_end_time, gps_start_time, gps_end_time, tag_imrppv2, tag_imrppv2, tag_imrppv2, tag_imrppv2)

os.system(pycbc_generate_hwinj_command_sxs)
os.system(pycbc_generate_hwinj_command_imrppv2)

In [ ]:
hw_inj_start_time_sxs = int(glob.glob('%s_*.xml.gz'%tag_sxs)[0][-17:-7])
hw_inj_start_time_imrppv2 = int(glob.glob('%s_*.xml.gz'%tag_imrppv2)[0][-17:-7])
print hw_inj_start_time_sxs, hw_inj_start_time_imrppv2

In [ ]:
tag, hw_inj_start_time =  tag_sxs, hw_inj_start_time_sxs

pycbc_insert_frame_hwinj_h1 = 'pycbc_insert_frame_hwinj --fake-strain zeroNoise --low-frequency-cutoff 25 --gps-start-time %d --gps-end-time %d --pad-data 8 --sample-rate 16384 --hwinj-file %s_%d_H1.txt  --hwinj-start-time %d --ifo H1 --output-file %s_H-H1HWINJ.gwf --strain-high-pass 1'%(gps_start_time, gps_end_time, tag, hw_inj_start_time,  hw_inj_start_time, tag)
pycbc_insert_frame_hwinj_l1 = 'pycbc_insert_frame_hwinj --fake-strain zeroNoise --low-frequency-cutoff 25 --gps-start-time %d --gps-end-time %d --pad-data 8 --sample-rate 16384 --hwinj-file %s_%d_L1.txt  --hwinj-start-time %d --ifo L1 --output-file %s_L-L1HWINJ.gwf --strain-high-pass 1'%(gps_start_time,gps_end_time, tag, hw_inj_start_time, hw_inj_start_time, tag)
pycbc_insert_frame_hwinj_v1 = 'pycbc_insert_frame_hwinj --fake-strain zeroNoise --low-frequency-cutoff 25 --gps-start-time %d --gps-end-time %d --pad-data 8 --sample-rate 16384 --hwinj-file %s_%d_L1.txt  --hwinj-start-time %d --ifo V1 --output-file %s_V-V1HWINJ.gwf --strain-high-pass 1'%(gps_start_time,gps_end_time, tag, hw_inj_start_time, hw_inj_start_time, tag)

os.system(pycbc_insert_frame_hwinj_h1)
print '... ran pycbc_insert_frame_hwinj for H1: ', pycbc_insert_frame_hwinj_h1

os.system(pycbc_insert_frame_hwinj_l1)
print '... ran pycbc_insert_frame_hwinj for L1: ', pycbc_insert_frame_hwinj_l1

os.system(pycbc_insert_frame_hwinj_v1)
print '... ran pycbc_insert_frame_hwinj for V1: ', pycbc_insert_frame_hwinj_v1

# Investigations

In [ ]:
tag_imrppv2 = 'SXS_BBH_0202_o2_HLV_1177532009_imrppv2'
tag_sxs = 'SXS_BBH_0202_o2_HLV_1177532009_sxs'

gps_start_time, gps_end_time, geocentric_end_time = 1173178789, 1173178917, 1173178853.0

h1_inj_imrppv2 = glob.glob('%s_*_H1.txt'%(tag_imrppv2))[0]
h1_inj_imrppv2 = np.genfromtxt(h1_inj_imrppv2)
h1_inj_sxs = glob.glob('%s_*_H1.txt'%(tag_sxs))[0]
h1_inj_sxs = np.genfromtxt(h1_inj_sxs)

h1_hoft_hwinj_imrppv2 = Fr.read_frame('%s_H-H1HWINJ.gwf'%tag_imrppv2, 'H1:HWINJ_INJECTED', gps_start_time, gps_end_time)
h1_hoft_hwinj_sxs = Fr.read_frame('%s_H-H1HWINJ.gwf'%tag_sxs, 'H1:HWINJ_INJECTED', gps_start_time, gps_end_time)

h1_freq_imrppv2, h1_psd_imrppv2 = np.genfromtxt('%s_H1_psd.txt'%tag_imrppv2, unpack=True)
h1_freq_sxs, h1_psd_sxs = np.genfromtxt('%s_H1_psd.txt'%tag_sxs, unpack=True)

plt.figure(figsize=(15,5))
plt.subplot(131)
plt.plot(h1_inj_imrppv2, alpha=0.2, color='r')
plt.plot(h1_inj_sxs, alpha=0.2, color='k', ls='dashed')

plt.subplot(132)
plt.plot(h1_hoft_hwinj_imrppv2.sample_times, h1_hoft_hwinj_imrppv2, alpha=0.2, color='r')
plt.plot(h1_hoft_hwinj_sxs.sample_times, h1_hoft_hwinj_sxs, alpha=0.2, color='k', ls='dashed')
plt.axvline(x=geocentric_end_time)
plt.xlim([geocentric_end_time-1, geocentric_end_time+0.1])
plt.hold(True)

plt.subplot(133)
plt.loglog(h1_freq_imrppv2, h1_psd_imrppv2,alpha=0.2, color='r')
plt.loglog(h1_freq_sxs, h1_psd_sxs, alpha=0.2, color='k', ls='dashed')
plt.xlim([25,1024])

plt.show()